In [1]:
# ======================================================
# [Step 0] 필수 라이브러리 설치 (패키지 충돌 최소화 적용)
# 기본 requests를 固定하면서 핵심 패키지를 설치합니다.
# ======================================================
!pip install -q requests==2.32.4
!pip install -q -U \
  langchain langchain-core langchain-community langchain-openai \
  langchain-chroma langchain-text-splitters langchain-huggingface \
  chromadb ragas datasets \
  sentence-transformers tiktoken gdown kiwipiepy rank_bm25
!pip uninstall -y langchain langchain-community langchain-core 2>/dev/null || true
!pip install -U -q langchain langchain-community langchain-core


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.2/502.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [2]:
import os, glob, random, time, gc, json, ast, re
import numpy as np
import torch
import pandas as pd
import gdown
from concurrent.futures import ThreadPoolExecutor

# AI & Vector DB
from langchain_chroma import Chroma              # ← langchain_community 대신 최신 패키지
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from rank_bm25 import BM25Okapi                 # ← BM25 직접 사용 (v3+ 방식)
from kiwipiepy import Kiwi
from pathlib import Path

# OpenAI & LangChain Core
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

# Evaluation (RAGAS)
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

# 보안 (API 키 입력용)
from getpass import getpass


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-1837334832.py:24: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipython-input-1837334832.py:24: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevanc

In [3]:
SEED = 42
CHUNK_SIZE = 800        # 문서 청크 분할 기준 글자 수
CHUNK_OVERLAP = 0       # 청크 중복 최소화
BATCH_SIZE = 800

# GPU 최적화 (RTX 2070 / Colab T4)
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.set_float32_matmul_precision('high')

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)
print('✅ 시드 고정 완료!')


In [5]:
# ======================================================
# [Step 1] Google Drive 마운트 및 DB 불러오기
# ======================================================
from google.colab import drive
import shutil

drive.mount('/content/drive')

# ⚠️ 사용자 드라이브 내 ChromaDB 경로로 수정하세요
drive_db_path = '/content/drive/MyDrive/chroma_db_combined_1771477980'
local_db_path = './chroma_db_combined_1771477980'

if os.path.exists(drive_db_path):
    if not os.path.exists(local_db_path):
        shutil.copytree(drive_db_path, local_db_path)
        print(f'✅ DB 복사 완료: {drive_db_path} → {local_db_path}')
    else:
        print(f'✅ 로컬 DB 이미 존재: {local_db_path}')
else:
    print(f'❌ Drive 경로를 찾을 수 없습니다: {drive_db_path}')
    print('경로를 확인하고 drive_db_path 변수를 수정해주세요.')


Mounted at /content/drive
✅ 성공: 드라이브의 DB를 코랩으로 가져왔습니다. (./chroma_db_combined_stored)


In [6]:
# ======================================================
# [Step 2] 전처리 유틸리티 함수 (processor.py 핵심 내용)
# ======================================================

def clean_json_to_text(raw_content: str) -> str:
    """벡터 DB에 저장된 JSON 형식 문자열을 사람이 읽기 쉬운 텍스트로 변환합니다."""
    content = raw_content.replace('passage: ', '')
    try:
        data = ast.literal_eval(content)
        if isinstance(data, dict):
            items = []
            for k, v in data.items():
                if v:
                    v_str = ', '.join(map(str, v)) if isinstance(v, list) else str(v)
                    items.append(f'{k}: {v_str}')
            return ', '.join(items)
        return content
    except Exception:
        return content


def get_clean_doc_text(doc: Document) -> str:
    """Document 정제 텍스트를 메타데이터에 캐시하여 중복 전처리를 방지합니다."""
    metadata = doc.metadata or {}
    clean_text = metadata.get('_clean_text')
    if clean_text is None:
        clean_text = clean_json_to_text(doc.page_content)
        doc.metadata = {**metadata, '_clean_text': clean_text}
    return clean_text


def get_kiwi_tokenizer() -> Kiwi:
    """Kiwi 멀티코어 형태소 분석기를 초기화합니다."""
    return Kiwi(num_workers=4)


TARGET_TAGS = ('N', 'V', 'X', 'S', 'VA')

def tokenize_corpus(docs: list, kiwi: Kiwi) -> list:
    """Document 리스트를 BM25용 토큰 코퍼스로 변환합니다 (형태소 분석, 배치 처리)."""
    clean_texts = [d.page_content.replace('passage: ', '') for d in docs]
    tokenized_corpus = []
    for tokens in kiwi.tokenize(clean_texts):
        tokenized_corpus.append([t.form for t in tokens if t.tag.startswith(TARGET_TAGS)])
    return tokenized_corpus


def clear_gpu() -> None:
    """GPU 캐시를 비워 메모리 누수를 방지합니다."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()


print('✅ 전처리 유틸리티 함수 정의 완료!')


🧬 BGE-M3 임베딩 모델 로드 중...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [7]:
# ======================================================
# [Step 3] OpenAI API 키 설정
# ======================================================
OPENAI_API_KEY = getpass('🔑 OpenAI API 키를 입력하세요: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print('✅ API 키 설정 완료!')


/tmp/ipython-input-902316399.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=LOAD_PATH, embedding_function=embeddings)


✅ 벡터 DB 로드 완료: 16150개


In [8]:
# ======================================================
# [Step 4-1] BGE-M3-ko 임베딩 모델 로드
# ======================================================

DB_PATH = './chroma_db_combined_1771477980'
EMBEDDING_MODEL_NAME = 'dragonkue/BGE-m3-ko'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'⚡ 디바이스: {device}')

encode_kwargs = {
    'normalize_embeddings': True,
    'batch_size': 128 if device == 'cuda' else 32,
}
model_kwargs = {'device': device}
if device == 'cuda':
    model_kwargs['model_kwargs'] = {'dtype': torch.float16}

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
print(f'✅ 임베딩 모델 로드 완료 (model={EMBEDDING_MODEL_NAME}, device={device})')

# ── ChromaDB 로드 ──────────────────────────────
vector_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
)
print(f'✅ ChromaDB 로드 완료: {vector_db._collection.count()} 문서')


OpenAI API Key: ··········


In [ ]:
# ======================================================
# [Step 4-2] CrossEncoder 리랭킹 모델 로드 (BAAI/bge-reranker-v2-m3)
# ======================================================

RERANKER_MODEL_NAME = 'BAAI/bge-reranker-v2-m3'
RERANK_MAX_LENGTH = 512

reranker = CrossEncoder(
    RERANKER_MODEL_NAME,
    device=device,
    max_length=RERANK_MAX_LENGTH,
    model_kwargs={'dtype': torch.float16 if device == 'cuda' else torch.float32},
)
print(f'✅ 리랭커 로드 완료 (model={RERANKER_MODEL_NAME}, device={device})')


🚀 16150개 문서 BM25 인덱싱 시작...


In [ ]:
# ======================================================
# [Step 4-3] Kiwi + BM25Okapi 인덱스 구축
#  rank_bm25를 직접 사용 (LangChain BM25Retriever 래퍼 오버헤드 제거)
# ======================================================

print('🔨 BM25 인덱스 구축 중...')
t0 = time.time()

# 1. 전체 DB 데이터 추출
all_data = vector_db.get()
bm25_docs = [
    Document(page_content=txt, metadata=meta)
    for txt, meta in zip(all_data['documents'], all_data['metadatas'])
]
print(f'   DB 추출 완료: {len(bm25_docs)}개 문서')

# 2. Kiwi 형태소 분석
kiwi = get_kiwi_tokenizer()
tokenized_corpus = tokenize_corpus(bm25_docs, kiwi)
print(f'   Kiwi 토크나이징 완료: {time.time()-t0:.2f}s')

# 3. BM25Okapi 직접 구축
bm25 = BM25Okapi(tokenized_corpus)
print(f'✅ BM25 인덱스 구축 완료! 총 소요: {time.time()-t0:.2f}s')


In [ ]:
# ======================================================
# [Step 5] 앙상블 검색 함수 (BM25 + Vector MMR + RRF 통합)
#  retriever.py의 get_ensemble_results 로직 기반
# ======================================================

# 증상 → 검색어 경량 확장 사전
_QUERY_HINTS = {
    '눈': ['안구건조', '눈 피로', '루테인', '오메가3'],
    '침침': ['시야흐림', '눈 피로', '루테인', '오메가3'],
    '건조': ['안구건조', '인공눈물', '오메가3'],
    '시야': ['시야흐림', '루테인'],
}

def _expand_query(query: str) -> str:
    """질의 증상 키워드에 검색 힌트를 추가하여 리콜을 보강합니다."""
    q = (query or '').strip()
    q_norm = q.lower()
    terms = [q]
    for key, hints in _QUERY_HINTS.items():
        if key in q_norm:
            terms.extend(hints)
    terms.extend(re.findall(r'[가-힣a-zA-Z0-9\-]{2,}', q))
    seen, dedup = set(), []
    for t in terms:
        k = t.strip().lower()
        if k and k not in seen:
            seen.add(k)
            dedup.append(t.strip())
    return ' '.join(dedup)


def _doc_key(doc: Document) -> str:
    """메타데이터 기반 문서 고유 키 (RRF 중복 제거용)."""
    meta = doc.metadata or {}
    return meta.get('id') or meta.get('_source_file') or meta.get('source') or str(hash(doc.page_content))


def get_ensemble_results(
    query: str,
    kiwi,
    bm25_retriever: BM25Okapi,
    vector_db,
    bm25_docs: list,
    k: int = 20,
    weight_bm25: float = 0.8,
    weight_vector: float = 0.2,
) -> list:
    """
    앙상블 검색: BM25(Kiwi 형태소) + Vector MMR → RRF 통합.
    weight_bm25=0.8 키워드 질문, weight_vector=0.8 의미 기반 질문에 적합.
    """
    # 1. 쿼리 확장
    search_keywords = _expand_query(query)

    # 2. Kiwi 토큰화 (TAG 필터)
    query_tokens = [
        t.form for t in kiwi.tokenize(search_keywords)
        if t.tag.startswith(('N', 'X', 'S', 'VA'))
    ]

    # 3. BM25 + Vector 병렬 검색
    def _bm25_search():
        if not query_tokens:
            return []
        scores = bm25_retriever.get_scores(query_tokens)
        top_idx = np.argsort(scores)[::-1][:k]
        return [bm25_docs[i] for i in top_idx if i < len(bm25_docs)]

    def _vector_search():
        return vector_db.max_marginal_relevance_search(
            f'query: {query}', k=k, fetch_k=min(k * 2, 40), lambda_mult=0.5
        )

    with ThreadPoolExecutor(max_workers=2) as ex:
        f_bm25 = ex.submit(_bm25_search)
        f_vec  = ex.submit(_vector_search)
        keyword_docs = f_bm25.result()
        vector_docs  = f_vec.result()

    # 4. RRF 통합 (메타데이터 기반 중복 제거)
    rrf_scores: dict = {}
    doc_map: dict    = {}

    def _add_rrf(docs, weight):
        for rank, doc in enumerate(docs):
            if not isinstance(doc, Document):
                continue
            key = _doc_key(doc)
            if key not in rrf_scores:
                rrf_scores[key] = 0.0
                doc_map[key] = doc
            rrf_scores[key] += weight * (1.0 / (rank + 60))

    _add_rrf(keyword_docs, weight_bm25)
    _add_rrf(vector_docs,  weight_vector)

    ranked_keys = sorted(rrf_scores, key=lambda k: rrf_scores[k], reverse=True)[:k]
    return [doc_map[k] for k in ranked_keys]


print('✅ 앙상블 검색 함수 정의 완료!')


In [ ]:
# ======================================================
# [Step 5-2] CrossEncoder 리랭킹 함수
#  retriever.py rerank_docs 기반 — GPU fp16 AMP 적용
# ======================================================

MIN_RERANK_SCORE = 0.005  # 관련성 낮은 문서 제거 임계값

def rerank_docs(
    query: str,
    docs: list,
    reranker: CrossEncoder,
    top_k: int = 5,
    batch_size: int = 32,
) -> list:
    """
    CrossEncoder로 앙상블 결과를 정밀 리랭킹합니다.
    Returns: [(score, doc), ...] 리스트 (상위 top_k)
    """
    pairs = [[query, clean_json_to_text(doc.page_content)] for doc in docs]

    with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
        scores = reranker.predict(
            pairs,
            batch_size=batch_size,
            show_progress_bar=False,
        )

    reranked = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
    top_items = list(reranked[:top_k])

    # MIN_RERANK_SCORE 필터 적용
    filtered = [(s, d) for s, d in top_items if s >= MIN_RERANK_SCORE]
    return filtered if filtered else top_items[:1]  # 최소 1개 유지


print('✅ 리랭킹 함수 정의 완료!')


In [ ]:
# ======================================================
# [Step 5-3] 컨텍스트 구성 함수 (generator.py build_context 기반)
# ======================================================

def build_context(final_docs: list, max_chars: int = 1000) -> str:
    """
    최종 선택 문서들을 LLM 컨텍스트 문자열로 변환합니다.
    각 문서는 최대 max_chars 글자로 제한합니다.
    """
    parts = []
    for i, doc in enumerate(final_docs, 1):
        source = os.path.basename(doc.metadata.get('source', 'Unknown'))
        content = get_clean_doc_text(doc)
        parts.append(f'[문서 {i}] (출처: {source})\n{content[:max_chars]}')
    return '\n\n'.join(parts)


print('✅ 컨텍스트 구성 함수 정의 완료!')


In [ ]:
# ======================================================
# [Step 6-1] 검색 쿼리 설정 및 앙상블 검색 실행
# ======================================================

user_query = '눈이 침침하고 눈물이 자주 나와요. 어떤 영양제가 좋을까요?'  # 💬 여기서 쿼리 수정

ENSEMBLE_K   = 20    # 앙상블 검색 후보 문서 수
TOP_K        = 5     # 리랭킹 후 최종 문서 수
WEIGHT_BM25  = 0.8   # BM25 가중치 (키워드 위주 질문)
WEIGHT_VEC   = 0.2   # Vector 가중치

print(f'🔍 앙상블 검색 중... (query: "{user_query}")')
t_search = time.time()

ensemble_docs = get_ensemble_results(
    query=user_query,
    kiwi=kiwi,
    bm25_retriever=bm25,
    vector_db=vector_db,
    bm25_docs=bm25_docs,
    k=ENSEMBLE_K,
    weight_bm25=WEIGHT_BM25,
    weight_vector=WEIGHT_VEC,
)

print(f'✅ 앙상블 검색 완료! {len(ensemble_docs)}개 문서 후보 (소요: {time.time()-t_search:.2f}s)')


In [ ]:
# ======================================================
# [Step 6-2] CrossEncoder 리랭킹 실행
# ======================================================

print(f'⚡ {len(ensemble_docs)}개 문서 리랭킹 중...')
t_rerank = time.time()

ranked_pairs = rerank_docs(
    query=user_query,
    docs=ensemble_docs,
    reranker=reranker,
    top_k=TOP_K,
    batch_size=32,
)

rerank_scores = [s for s, _ in ranked_pairs]
final_docs    = [d for _, d in ranked_pairs]
context_text  = build_context(final_docs)

print(f'✅ 리랭킹 완료! 최종 {len(final_docs)}개 문서 선택 (소요: {time.time()-t_rerank:.2f}s)')
print('\n[선택된 문서 목록]')
for i, (score, doc) in enumerate(ranked_pairs, 1):
    filename = os.path.basename(doc.metadata.get('source', 'Unknown'))
    preview  = doc.page_content.replace('passage: ', '').replace('\n', ' ')[:80]
    print(f'  [{i}] score={score:.4f} | {filename} | "{preview}..."')


In [ ]:
# ======================================================
# [Step 6-3] GPT 답변 생성 (generator.py 프롬프트 기반)
# ======================================================

_ANSWER_PROMPT_TEMPLATE = """\
당신은 공인된 전문 약사입니다.
제공된 [검색된 문서]에 근거하여 답변하십시오.

⚠️ 지침:
1. 모든 답변은 [검색된 문서]에 기재된 내용만 사용하십시오. 문서에 없는 성분명, 용량, 질환명, 상호작용 정보를 직접 추가하거나 추측하지 마십시오.
2. 문서에 부작용의 강도가 전혀 언급되지 않았다면, 임의로 등급을 매기지 마십시오.
3. 답변의 각 정보 뒤에 반드시 해당 근거가 된 **[문서 N]**을 표기하십시오.
4. 근거 문서를 제시하기 전에 "자세한 내용은 전문가와 꼭 상담하세요."라는 문구를 포함하십시오.
5. 질문에 대한 직접적인 답이 문서에 없더라도, 관련 문서 내용이 있으면 해당 문서 내용을 그대로 인용하여 답변하십시오.
6. 답변의 첫 문장에서 질문의 핵심 키워드(약품명, 증상명 등)를 포함하여 질문에 직접 답하십시오.

[검색된 문서]
{context}

[질문]
{question}

[답변]"""

gen_llm = ChatOpenAI(model='gpt-5.1', temperature=0.1, api_key=OPENAI_API_KEY)
prompt_template = PromptTemplate.from_template(_ANSWER_PROMPT_TEMPLATE)
chain = prompt_template | gen_llm | StrOutputParser()

print('✍️ 답변 생성 중...')
t_gen = time.time()

final_answer = chain.invoke({'context': context_text, 'question': user_query})

print(f'✅ 생성 완료! (소요: {time.time()-t_gen:.2f}s)')
print('=' * 60)
print(f'\n[AI 약사 답변]\n{final_answer}')
print('\n' + '-' * 60)
print('[근거 문서 (Evidence)]')
for i, doc in enumerate(final_docs, 1):
    filename = os.path.basename(doc.metadata.get('source', 'Unknown'))
    preview  = doc.page_content.replace('passage: ', '').replace('\n', ' ')[:100]
    print(f'[문서 {i}] {filename}\n   "{preview}..."')


In [ ]:
# ======================================================
# [Step 7] 자기 검증 (Verification) — generator.py 기반
# ======================================================

_VERIFY_PROMPT_TEMPLATE = """\
당신은 '식품의약품 안전처', 미국 'FDA' 등 공신력 있는 허가 기관의 엄격한 감독관입니다.
당신의 역할은 전문약사가 작성한 [검증 대상 답변]이 [검색된 문서]의 내용을 바탕으로 **논리적으로 타당한지** 평가하는 것입니다.

[검색된 문서 (Ground Truth)]
{context}

[질문]
{question}

[검증 대상 답변]
{answer}

[평가 기준]
1. 논리적 비약: 문서에 직접적인 단어가 없더라도, 문서 내용으로부터 합리적으로 유추한 것이라면 PASS로 판정하십시오.
2. 환각(Hallucination): 문서에 전혀 없는 내용을 근거 없이 지어냈을 때만 FAIL로 판정하십시오.
3. 안전 권고: "전문가와 상담하세요" 등 환자 안전을 위한 기본 권고는 PASS로 허용합니다.

[출력 형식]
반드시 아래 형식을 지켜주세요.
- [분석 코멘트]: (근거와 주장의 연결고리가 타당한지 설명)
- [최종 판정]: PASS 또는 FAIL"""

verifier_llm = ChatOpenAI(model='gpt-5.2', temperature=0.0, api_key=OPENAI_API_KEY)
verify_prompt = PromptTemplate.from_template(_VERIFY_PROMPT_TEMPLATE)
verify_chain  = verify_prompt | verifier_llm | StrOutputParser()

verify_result = verify_chain.invoke({
    'context':  context_text,
    'question': user_query,
    'answer':   final_answer,
})

print('\n🔍 [GPT-5.2 검증 결과]')
print('-' * 40)
print(verify_result)
print('-' * 40)

def _is_pass(vr: str) -> bool:
    """[최종 판정] 패턴에서 PASS/FAIL을 파싱합니다."""
    m = re.search(r'\[최종\s*판정\]\s*[:：]\s*(PASS|FAIL)', vr, re.IGNORECASE)
    if m:
        return m.group(1).upper() == 'PASS'
    tokens = re.findall(r'\b(PASS|FAIL)\b', vr, re.IGNORECASE)
    return tokens[-1].upper() == 'PASS' if tokens else False

if _is_pass(verify_result):
    print('\n✅ 검증 통과! (최종 답변을 사용합니다)')
else:
    print('\n❌ 검증 실패(FAIL)! (다음 단계에서 자동 교정이 실행됩니다)')


In [ ]:
# ======================================================
# [Step 8] RAGAS 평가 (generator.py evaluate_with_ragas 기반)
#  answer_relevancy: BGE-M3 대신 OpenAI text-embedding-3-small 사용
# ======================================================

ragas_data = {
    'question': [user_query],
    'answer':   [final_answer],
    'contexts': [[d.page_content.replace('passage: ', '')[:1000] for d in final_docs]],
}
dataset = Dataset.from_dict(ragas_data)

eval_llm        = ChatOpenAI(model='gpt-5.2', api_key=OPENAI_API_KEY)
ragas_embeddings = OpenAIEmbeddings(model='text-embedding-3-small', api_key=OPENAI_API_KEY)

print('📊 RAGAS 평가 중 (GPT-5.2 + text-embedding-3-small)...')

try:
    results   = evaluate(
        dataset=dataset,
        metrics=[faithfulness, answer_relevancy],
        llm=eval_llm,
        embeddings=ragas_embeddings,
        raise_exceptions=False,
    )
    df_result = results.to_pandas()

    def _safe_metric(df, keywords):
        for col in df.columns:
            if any(k.lower() in col.lower() for k in keywords):
                v = df.iloc[0][col]
                try:
                    import math
                    f = float(v)
                    return 0.0 if (math.isnan(f) or math.isinf(f)) else max(0.0, min(1.0, f))
                except Exception:
                    return 0.0
        return 0.0

    f_val = _safe_metric(df_result, ['faithfulness'])
    r_val = _safe_metric(df_result, ['relevancy', 'relevance'])

    print('\n' + '=' * 50)
    print('📊 [RAG Evaluation Report]')
    print(f'🤖 평가 모델: GPT-5.2 / text-embedding-3-small')
    print('-' * 50)
    print(f'✅ 신뢰성 (Faithfulness)      : {f_val:.4f}')
    print(f'📎 답변 적절성 (Answer Relevancy): {r_val:.4f}')
    print('=' * 50)

except Exception as e:
    print(f'❌ RAGAS 평가 중 오류 발생: {e}')


In [ ]:
# ======================================================
# [Step 9] 자기 교정 루프 (Self-Correction Loop)
#  generator.py self_correction_loop 동기 Colab용 버전
# ======================================================

_CORRECTION_PROMPT_TEMPLATE = """\
당신은 검증 피드백을 바탕으로 답변을 수정하는 **전문 약사**입니다.

[사용자 질문]: {question}
[검색된 문서]: {context}
[이전 답변]: {answer}
[검증 피드백]: {verify_result}

위 피드백을 반영하여 오류를 바로잡고, 다시 최선의 답변을 작성하십시오.
주의사항:
- [검색된 문서]에 관련 정보가 있다면 해당 정보를 활용하여 답변하십시오.
- 모든 정보 뒤에 [문서 N] 출처를 표기하십시오.
- "자세한 내용은 전문가와 꼭 상담하세요."를 포함하십시오.
- 수정된 답변만 출력하십시오."""

_OPTIMIZER_PROMPT_TEMPLATE = """\
당신은 'RAG 시스템 프롬프트 엔지니어링 전문가'입니다.
이전 라운드에서 생성된 답변이 검증 실패 판정을 받았습니다.

[사용자 질문]: {question}
[검증 결과]: {verify_result}
[기존 프롬프트 템플릿]: {original_template}

위의 실패 원인을 분석하여, 다음 라운드에서 더 정확한 답변을 생성할 수 있도록 수정된 프롬프트 템플릿을 만드십시오.
- [검색된 문서]의 데이터를 더 정확하게 인용하고 추측을 배제하도록 지시를 강화하세요.
- 반드시 {{context}}와 {{question}} 변수를 포함한 전체 프롬프트 전문만 출력하세요."""


MAX_ROUNDS     = 2
is_passed      = _is_pass(verify_result)
current_answer = final_answer
current_template = _ANSWER_PROMPT_TEMPLATE
last_verify    = verify_result

print(f'🤖 자기 교정 루프 시작 (최대 {MAX_ROUNDS}회)')

for round_num in range(1, MAX_ROUNDS + 1):
    if _is_pass(last_verify):
        print(f'✅ PASS 판정 — 교정 종료 (round {round_num-1})')
        break

    print(f'\n--- [ROUND {round_num} 교정 시작] ---')

    # 1. 프롬프트 최적화 (gpt-4o-mini)
    opt_llm   = ChatOpenAI(model='gpt-4o-mini', temperature=0.2, api_key=OPENAI_API_KEY)
    opt_chain = PromptTemplate.from_template(_OPTIMIZER_PROMPT_TEMPLATE) | opt_llm | StrOutputParser()
    current_template = opt_chain.invoke({
        'question': user_query,
        'verify_result': last_verify,
        'original_template': current_template,
    })
    print(f'⚙️  Round {round_num}: 프롬프트 최적화 완료')

    # 2. 교정 답변 재생성
    corr_llm   = ChatOpenAI(model='gpt-5.1', temperature=0.1, api_key=OPENAI_API_KEY)
    corr_chain = PromptTemplate.from_template(_CORRECTION_PROMPT_TEMPLATE) | corr_llm | StrOutputParser()
    current_answer = corr_chain.invoke({
        'question': user_query,
        'context':  context_text,
        'answer':   current_answer,
        'verify_result': last_verify,
    })
    print(f'✍️  Round {round_num}: 교정 답변 생성 완료')

    # 3. 재검증
    last_verify = verify_chain.invoke({
        'context':  context_text,
        'question': user_query,
        'answer':   current_answer,
    })
    print(f'🔍 Round {round_num} 검증 결과: {"PASS" if _is_pass(last_verify) else "FAIL"}')

else:
    if not _is_pass(last_verify):
        print(f'\n⚠️  최대 교정 횟수({MAX_ROUNDS}회) 도달. 마지막 답변을 사용합니다.')

print('\n' + '=' * 60)
print('[🏁 최종 확정 답변]')
print(current_answer)


In [ ]:
# ======================================================
# [Step 10] 전체 파이프라인 결과 요약
# ======================================================

print('\n' + '🏆' * 20)
print('[ 약사 RAG 파이프라인 실행 결과 요약 ]')
print('🏆' * 20)
print(f'\n🔍 검색 쿼리    : {user_query}')
print(f'📄 앙상블 후보 : {len(ensemble_docs)}개')
print(f'🎯 리랭킹 최종 : {len(final_docs)}개')
print(f'📑 리랭킹 점수 : {[round(s, 4) for s in rerank_scores]}')
print()
print('=' * 60)
print('[🤖 최종 AI 약사 답변]')
print(current_answer)
print()
print(f'📋 검증 결과   : {"✅ PASS" if _is_pass(last_verify) else "❌ FAIL"}')
print('='*60)
